In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import seaborn as sns
import io #Used as buffer

# Functions

In [2]:
path= Path(f'../../data/cluster_results')
col_by_freq = True
col_palette = 'Set3'
pat = re.compile('.*/(exp\d)/(\w*)/nonn/freq([0-9\.]+)div(\w+)/(\w+)_\w+.csv')
decode_keys = ['experiment','function','freq','div','method']
nn_pat = re.compile('.*/(exp\d)/(\w*)/nn/freq([0-9\.]+)\w+div([A-Za-z]+)/(\w+)_(\w+)_\w+.csv')
nn_decode_keys = ['experiment','function','freq','div','method','replace_mech']

def get_nn_files(m): return list(path.glob(f'**/nn/**/*{m}.csv'))


def read_nn_csv(f,m):
    df = pd.read_csv(f)
    for k,v in zip(nn_decode_keys,nn_pat.search(str(f)).groups()): df[k] = v
    df['freq'] = df['freq'].astype(float)
    df['method'] = df['method'].str.replace('NNnorm_Worst', 'NN')
    df['method'] = df['method'] + '_' + df['div'] #'_' + df['replace_mech'] +
    
    df.drop(['replace_mech','div'], axis=1, inplace=True)
    df.rename({'0':m.upper(), m:m.upper()}, axis=1, inplace=True)
    return df

def get_data(m, normalize=False):
    nn_files = get_nn_files(m)
    data = pd.concat([read_nn_csv(f,m) for f in nn_files])
    if normalize:
        data_norm = (data.groupby(['experiment','function','freq','method'])[m.upper()].mean().reset_index()
                         .groupby(['experiment','function'])[m.upper()].min().reset_index()
                         .rename({m.upper():m.upper()+'_norm'}, axis=1))
        data = data.merge(data_norm, 'left')
        data[m.upper()+'_norm'] = data[m.upper()] / data[m.upper()+'_norm']
    return data




# Table

In [3]:
df=get_data('nn_time', normalize=False)
df['function']=df['function'].str.title()
df.head()

,NN_TIME,experiment,function,freq,method
0,0.097175,exp1,Sphere,1.0,NN_HMu
1,0.096994,exp1,Sphere,1.0,NN_HMu
2,0.096883,exp1,Sphere,1.0,NN_HMu
3,0.096573,exp1,Sphere,1.0,NN_HMu
4,0.096567,exp1,Sphere,1.0,NN_HMu


In [5]:
table2=df.pivot_table(values='NN_TIME', index=['function','method'] ,columns=['experiment','freq'])

In [6]:
print(table2.to_csv(float_format='%.3f'))

experiment,,exp1,exp1,exp1,exp1,exp1,exp2,exp2,exp2,exp2,exp2,exp3,exp3,exp3,exp3,exp3,exp4,exp4,exp4,exp4,exp4
freq,,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0,0.5,1.0,4.0,10.0,20.0
function,method,,,,,,,,,,,,,,,,,,,,
Rastrigin,NN_CwN,0.194,0.097,0.024,0.010,0.005,0.197,0.099,0.025,0.010,0.005,0.200,0.099,0.025,0.010,0.005,0.189,0.094,0.023,0.010,0.005
Rastrigin,NN_HMu,0.192,0.098,0.024,0.010,0.007,0.194,0.097,0.025,0.010,0.005,0.201,0.136,0.025,0.010,0.007,0.192,0.099,0.024,0.010,0.005
Rastrigin,NN_No,0.197,0.099,0.024,0.010,0.005,0.196,0.098,0.025,0.010,0.005,0.201,0.101,0.024,0.010,0.005,0.196,0.095,0.024,0.010,0.005
Rastrigin,NN_RI,0.198,0.098,0.025,0.010,0.005,0.197,0.099,0.024,0.010,0.005,0.203,0.138,0.025,0.010,0.005,0.194,0.098,0.025,0.010,0.005
Rosenbrock,NN_CwN,0.196,0.098,0.024,0.010,0.005,0.192,0.097,0.024,0.010,0.005,0.200,0.100,0.025,0.010,0.005,0.193,0.095,0.023,0.009,0.005
Rosenbrock,NN_HMu,0.193,0.097,0.024,0.010,0.005,0.195,0.097,0.025,0.010,0.